In [1]:
import glob
import pandas as pd
import numpy as np
import json
import pickle
from bson.binary import Binary
from pymongo import MongoClient
import certifi
from ast import literal_eval

from pymatgen.core import Lattice, Structure
from crystal_out import crystalOut


# Uploading to mongodb in Orion

In [4]:
mc = MongoClient(host="orion.nus.edu.sg",username="raman_ml",password="ch4ndr4s3kh4r4")
db = mc["raman_ml"]
db

Database(MongoClient(host=['orion.nus.edu.sg:27017'], document_class=dict, tz_aware=False, connect=True), 'raman_ml')

In [ ]:
one_crystal_outputs = glob.glob("/crystal17_output_files/calc-Sb2O5_15_icsd1422_tzvp_PBE0/Sb2O5_15.out")
print("Found", len(one_crystal_outputs), "structures to be loaded into mongodb...")

#print("Structures identified to have errors:")
#print("popped:", all_crystal_outputs.pop(52))
#print("popped:", all_crystal_outputs.pop(52))

structures = db.structures
for s in one_crystal_outputs:
    selected_structure = crystalOut(s)
    name = s.split("\\")[-1]
    print("Working on", name)
    bornChargeArrayList = []
    for val in selected_structure.bornCharge.values():
        bornChargeArrayList.append(val["Born Charge"])
    bornChargeArray = np.concatenate(bornChargeArrayList, dtype = "float64")

    structure = {"structure_name": name,
                "structure":json.dumps(selected_structure.structure.as_dict()), 
                "spaceGroup":selected_structure.parsed_data["space_group"],
                "thermodynamicTerms":json.dumps(selected_structure.thermodynamicTerms),
                "dielectricTensor":Binary(pickle.dumps(selected_structure.dielectricTensor, protocol=2), subtype=128),
                "vibContributionsDielectric":Binary(pickle.dumps(selected_structure.vibContributionsDielectric, protocol=2), subtype=128),
                "secondElectricSusceptibility":Binary(pickle.dumps(selected_structure.secondElectricSusceptibility, protocol=2), subtype=128),
                "thirdElectricSusceptibility":Binary(pickle.dumps(selected_structure.thirdElectricSusceptibility, protocol=2), subtype=128),
                "bornChargeArray":Binary(pickle.dumps(bornChargeArray, protocol=2), subtype=128),
                "bornChargeNormalModeBasis":Binary(pickle.dumps(selected_structure.bornChargeNormalModeBasis, protocol=2), subtype=128),
                "intRaman":json.dumps(selected_structure.intRaman)}

    structure_id = structures.insert_one(structure).inserted_id
    print("Done", structure_id)

Found 62 structures to be loaded into mongodb...
Structures identified to have errors:
popped: ../project_Raman/crystal17_output_files\calc-Sb2O5_15_icsd1422_tzvp_PBE0\Sb2O5_15.out
popped: ../project_Raman/crystal17_output_files\calc-Sb2Se3_62_icsd30973_tzvp_PBE0\Sb2Se3_62.out
>>>>>>> Optimized structure found.
Doing As2O3_14.out
Done 62dfc6755c64517df4cfc762
Doing As2O3_14.out
Done 62dfc6755c64517df4cfc763
>>>>>>> Optimized structure found.
>>>>>>> CRYSTALLOGRAPHIC CELL found.
Doing As2O3_227.out
Done 62dfc6765c64517df4cfc764
>>>>>>> Optimized structure found.
Doing As2S3_2.out
Done 62dfc6765c64517df4cfc765
>>>>>>> Optimized structure found.
>>>>>>> CRYSTALLOGRAPHIC CELL found.
Doing As2Se3_12.out
Done 62dfc6765c64517df4cfc766
>>>>>>> Optimized structure found.
Doing As4S3_62.out
Done 62dfc6765c64517df4cfc767
>>>>>>> Optimized structure found.
Doing As4S5_11.out
Done 62dfc6775c64517df4cfc768
>>>>>>> Optimized structure found.
Doing As8S9_13.out
Done 62dfc6775c64517df4cfc769
>>>>>>> Op

In [10]:
# list orion mongo
from pymongo import MongoClient

mc = MongoClient(host="orion.nus.edu.sg",username="raman_ml",password="ch4ndr4s3kh4r4")
db = mc["raman_ml"]
structures = db.structures
cursor = structures.find({"structure_name":"As2Se3_12.out"})
for p in cursor:
    print(p)

{'_id': ObjectId('62dfc6765c64517df4cfc766'), 'structure_name': 'As2Se3_12.out', 'structure': '{"@module": "pymatgen.core.structure", "@class": "Structure", "charge": 0, "lattice": {"matrix": [[7.31519040398197, 0.0, -0.3573098538053958], [-6.378234105044253, 3.5819185289115985, 0.35730985380539676], [0.0, 0.0, 9.56464305]], "a": 7.323911590000001, "b": 7.32391159, "c": 9.56464305, "alpha": 87.203615, "beta": 92.79638499999999, "gamma": 150.717793, "volume": 250.61675657099957}, "sites": [{"species": [{"element": "As", "occu": 1}], "abc": [0.3849458405645, -0.3849458405645, -0.4334961641515], "xyz": [5.271226807833624, -1.3788446389454325, -4.421325957683497], "label": "As", "properties": {}}, {"species": [{"element": "As", "occu": 1}], "abc": [-0.3849458405645, 0.3849458405645, 0.4334961641515], "xyz": [-5.271226807833624, 1.3788446389454325, 4.421325957683497], "label": "As", "properties": {}}, {"species": [{"element": "As", "occu": 1}], "abc": [-0.2050146021549, 0.2050146021549, -0.

# Reading

In [11]:
def load_structure_from_mongo(structure_name):
    mc = MongoClient(host="orion.nus.edu.sg",username="raman_ml",password="ch4ndr4s3kh4r4")
    db = mc["raman_ml"]
    structures = db.structures
    structure_name_out = structure_name + ".out"
    query = structures.find_one({"structure_name":structure_name_out})

    structure = Structure.from_dict(json.loads(query["structure"]))
    spaceGroup = query["spaceGroup"]
    thermodynamicTerms = json.loads(query["thermodynamicTerms"])
    intRaman = json.loads(query["intRaman"])

    dielectricTensor = pickle.loads(query["dielectricTensor"])
    vibContributionsDielectric = pickle.loads(query["vibContributionsDielectric"])
    secondElectricSusceptibility = pickle.loads(query["secondElectricSusceptibility"])
    thirdElectricSusceptibility = pickle.loads(query["thirdElectricSusceptibility"])
    bornChargeArray = pickle.loads(query["bornChargeArray"])
    bornChargeNormalModeBasis = pickle.loads(query["bornChargeNormalModeBasis"])

    return structure, spaceGroup, thermodynamicTerms, intRaman, dielectricTensor, vibContributionsDielectric, secondElectricSusceptibility, thirdElectricSusceptibility, bornChargeArray, bornChargeNormalModeBasis



In [16]:
selected_name = "As2Se3_12"
structure, spaceGroup, thermodynamicTerms, intRaman, dielectricTensor, vibContributionsDielectric, secondElectricSusceptibility, thirdElectricSusceptibility, bornChargeArray, bornChargeNormalModeBasis = load_structure_from_mongo(selected_name)
print(thermodynamicTerms)
structure

{'EL': -635390.9295402009, 'ZPE': 0.25050851011, 'PV': 0.000158495294, 'TS': 1.03770079784, 'HEAT CAPACITY': 2.207902560284}


Structure Summary
Lattice
    abc : 7.323911590000001 7.32391159 9.56464305
 angles : 87.203615 92.79638499999999 150.717793
 volume : 250.61675657099957
      A : 7.31519040398197 0.0 -0.3573098538053958
      B : -6.378234105044253 3.5819185289115985 0.35730985380539676
      C : 0.0 0.0 9.56464305
PeriodicSite: As (5.2712, -1.3788, -4.4213) [0.3849, -0.3849, -0.4335]
PeriodicSite: As (-5.2712, 1.3788, 4.4213) [-0.3849, 0.3849, 0.4335]
PeriodicSite: As (-2.8074, 0.7343, -1.1738) [-0.2050, 0.2050, -0.1380]
PeriodicSite: As (2.8074, -0.7343, 1.1738) [0.2050, -0.2050, 0.1380]
PeriodicSite: Se (-0.4949, 0.1295, -2.7919) [-0.0361, 0.0361, -0.2946]
PeriodicSite: Se (0.4949, -0.1295, 2.7919) [0.0361, -0.0361, 0.2946]
PeriodicSite: Se (3.1922, -0.8350, -3.3299) [0.2331, -0.2331, -0.3307]
PeriodicSite: Se (-3.1922, 0.8350, 3.3299) [-0.2331, 0.2331, 0.3307]
PeriodicSite: Se (-4.9722, 1.3006, -0.0728) [-0.3631, 0.3631, -0.0347]
PeriodicSite: Se (4.9722, -1.3006, 0.0728) [0.3631, -0.3631, 0.0347